In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import cv2
import glob
from sklearn.model_selection import train_test_split
from torchvision import models
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import seaborn as sns
import pandas as pd
import random
from torch.nn import functional as F
import gc  # For explicit garbage collection
from sklearn.utils import resample


In [ ]:
# Set seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

# Define Constants
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_SIZE = 224
BATCH_SIZE = 8  # Reduced from 16 to save memory
EPOCHS = 100
PATIENCE = 10
LEARNING_RATE = 3e-4
NUM_FRAMES = 16  


In [3]:
# Dataset paths
VIOLENT_PATHS = [
    "/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence",
    "/kaggle/input/project-data/Complete Dataset/train/Fight",
    "/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Test/Violence",
    "/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Train/Violence",
    "/kaggle/input/project-data/Complete Dataset/val/Fight"
]

NON_VIOLENT_PATHS = [
    "/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence",
    "/kaggle/input/project-data/Complete Dataset/train/NonFight",
    "/kaggle/input/project-data/Complete Dataset/val/NonFight",
    "/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Test/Normal",
    "/kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Train/Normal"
]
TEST_SIZE = 0.2
RANDOM_STATE = 42

In [4]:
# Memory monitoring function
def print_gpu_memory():
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        reserved = torch.cuda.memory_reserved(0) / 1024**3
        allocated = torch.cuda.memory_allocated(0) / 1024**3
        free = total_memory - reserved

        print(f"GPU Memory: Total={total_memory:.2f}GB | Reserved={reserved:.2f}GB | Allocated={allocated:.2f}GB | Free={free:.2f}GB")

def clear_gpu_memory():
    """Clear GPU cache and run garbage collector"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import glob
import os

def prepare_dataset():
    """
    Prepare the dataset by gathering all video paths and creating labels.
    Returns:
        - Training and test splits (video paths and labels)
        - Separate hold-out test set (20 videos)
    """
    # List of common video extensions
    VIDEO_FORMATS = ['mp4', 'avi', 'mov', 'mkv', 'flv', 'wmv']
    
    # Gather violent videos and print counts per path
    violent_videos = []
    print("Violent videos per path:")
    for path in VIOLENT_PATHS:
        videos_in_path = []
        for ext in VIDEO_FORMATS:
            videos_in_path.extend(glob.glob(os.path.join(path, f"*.{ext}")))
        print(f"- {path}: {len(videos_in_path)} videos")
        violent_videos.extend(videos_in_path)
    
    # Gather non-violent videos and print counts per path
    non_violent_videos = []
    print("\nNon-violent videos per path:")
    for path in NON_VIOLENT_PATHS:
        videos_in_path = []
        for ext in VIDEO_FORMATS:
            videos_in_path.extend(glob.glob(os.path.join(path, f"*.{ext}")))
        print(f"- {path}: {len(videos_in_path)} videos")
        non_violent_videos.extend(videos_in_path)
    
    # Reserve 20 videos (10 violent + 10 non-violent) for separate testing
    holdout_violent = resample(violent_videos, replace=False, n_samples=10, random_state=42)
    holdout_non_violent = resample(non_violent_videos, replace=False, n_samples=10, random_state=42)
    holdout_paths = holdout_violent + holdout_non_violent
    holdout_labels = [1] * 10 + [0] * 10

    # Remove holdout videos from the main dataset
    violent_videos = [v for v in violent_videos if v not in holdout_violent]
    non_violent_videos = [v for v in non_violent_videos if v not in holdout_non_violent]

    # Balance the remaining dataset
    def balance_dataset(violent, non_violent):
        min_size = min(len(violent), len(non_violent))
        violent_balanced = resample(violent, replace=False, n_samples=min_size, random_state=42)
        non_violent_balanced = resample(non_violent, replace=False, n_samples=min_size, random_state=42)
        return violent_balanced, non_violent_balanced

    violent_videos, non_violent_videos = balance_dataset(violent_videos, non_violent_videos)

    # Create paths and labels for the main dataset
    all_video_paths = violent_videos + non_violent_videos
    all_labels = [1] * len(violent_videos) + [0] * len(non_violent_videos)

    # Print dataset statistics
    print(f"\nMain dataset after balancing:")
    print(f"- Violent videos: {len(violent_videos)}")
    print(f"- Non-violent videos: {len(non_violent_videos)}")
    print(f"- Total: {len(all_video_paths)} videos")
    print(f"\nHold-out test set: {len(holdout_paths)} videos (10 violent + 10 non-violent)")

    # Train-test split (stratified)
    video_paths_train, video_paths_test, labels_train, labels_test = train_test_split(
        all_video_paths, all_labels, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=all_labels
    )

    print(f"\nTraining set: {len(video_paths_train)} videos")
    print(f"Test set (main): {len(video_paths_test)} videos")

    return (
        video_paths_train, video_paths_test, labels_train, labels_test,
        holdout_paths, holdout_labels
    )

# Simplified data augmentation to reduce computation
class VideoAugmentation:
    def __init__(self, p=0.5):
        self.p = p

    def random_flip(self, frames):
        if random.random() < self.p:
            return torch.flip(frames, [3])  # Horizontal flip
        return frames

    def random_crop_resize(self, frames):
        if random.random() < self.p:
            h, w = frames.shape[2], frames.shape[3]
            new_h, new_w = int(h * 0.8), int(w * 0.8)

            # Random crop coordinates
            top = random.randint(0, h - new_h)
            left = random.randint(0, w - new_w)

            # Apply crop and resize to all frames at once using slicing
            cropped = frames[:, :, top:top+new_h, left:left+new_w]
            resized = F.interpolate(cropped, size=(h, w), mode='bilinear', align_corners=False)
            return resized
        return frames

    def __call__(self, frames):
        frames = self.random_flip(frames)
        frames = self.random_crop_resize(frames)
        return frames


In [5]:

class ViolenceDataset(Dataset):
    def __init__(self, video_paths, labels, transform=None, is_train=False):
        self.video_paths = video_paths
        self.labels = labels
        self.transform = transform
        self.is_train = is_train
        self.augmentation = VideoAugmentation() if is_train else None

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]

        # Open video
        cap = cv2.VideoCapture(video_path)
        frames = []
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Uniform sampling: choose frames at regular intervals
        if frame_count <= NUM_FRAMES:
            # If video has fewer frames than needed, duplicate frames
            indices = list(range(frame_count)) + [frame_count-1] * (NUM_FRAMES - frame_count)
        else:
            # Otherwise, sample uniformly
            indices = np.linspace(0, frame_count - 1, NUM_FRAMES, dtype=int)

        # Process each frame
        success = True
        frame_idx = 0

        while success and len(frames) < NUM_FRAMES:
            success, frame = cap.read()
            if not success:
                break

            if frame_idx in indices:
                # Preprocess the frame
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
                frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
                frame = torch.tensor(frame).permute(2, 0, 1).float() / 255.0
                frames.append(frame)

            frame_idx += 1

        cap.release()

        # Handle case of zero frames (corrupted video)
        if len(frames) == 0:
            # Create a blank frame
            blank_frame = torch.zeros(3, IMG_SIZE, IMG_SIZE)
            frames = [blank_frame] * NUM_FRAMES
        # Handle case of fewer frames than required
        elif len(frames) < NUM_FRAMES:
            # Duplicate the last frame to reach NUM_FRAMES
            last_frame = frames[-1]
            frames.extend([last_frame] * (NUM_FRAMES - len(frames)))

        # Stack frames and apply augmentation if in training mode
        frames = torch.stack(frames)
        if self.is_train and self.augmentation:
            frames = self.augmentation(frames)

        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return frames, label

# Custom collate function for batching
def custom_collate(batch):
    frames = torch.stack([item[0] for item in batch])  # Shape: [B, T, C, H, W]
    labels = torch.stack([item[1] for item in batch])
    return frames, labels

# Memory-efficient feature extractor using EfficientNet
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights

class FeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(FeatureExtractor, self).__init__()
        # Use ConvNeXt instead of EfficientNet
        weights = ConvNeXt_Tiny_Weights.IMAGENET1K_V1 if pretrained else None
        base_model = convnext_tiny(weights=weights)
        # Remove the classification layer
        self.model = nn.Sequential(*list(base_model.children())[:-1])
        self.feature_dim = 768  # ConvNeXt tiny output dimension

    def forward(self, x):
        batch_size, seq_len, c, h, w = x.shape

        # Process frames in chunks to save memory
        features = []
        chunk_size = 4  # Process 4 frames at a time

        for i in range(0, seq_len, chunk_size):
            end_idx = min(i + chunk_size, seq_len)
            chunk = x[:, i:end_idx].reshape(-1, c, h, w)  # Reshape chunk

            # Extract features
            with torch.cuda.amp.autocast(enabled=True):  # Use mixed precision
                chunk_features = self.model(chunk)

            # Reshape back
            chunk_features = chunk_features.reshape(batch_size, end_idx - i, -1)
            features.append(chunk_features)

            # Clear unnecessary tensors to free memory
            del chunk, chunk_features
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        # Concatenate chunks
        x = torch.cat(features, dim=1)
        return x

# Improved temporal modeling with reduced complexity
class TemporalAttention(nn.Module):
    def __init__(self, feature_dim, num_heads=4, dropout=0.1):  # Reduced heads
        super(TemporalAttention, self).__init__()
        self.self_attn = nn.MultiheadAttention(feature_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(feature_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Apply self-attention with residual connection
        attn_output, _ = self.self_attn(x, x, x)
        x = x + self.dropout(attn_output)
        x = self.norm1(x)
        return x

# Simplified sequence model
class TemporalSequenceModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=384, num_layers=1, dropout=0.5):  # Reduced complexity
        super(TemporalSequenceModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0)
        self.attention = TemporalAttention(hidden_dim)
        self.fc = nn.Linear(hidden_dim, 2)  # 2 classes
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(hidden_dim)

    def forward(self, x):
        # Process with LSTM
        lstm_out, _ = self.lstm(x)

        # Apply attention
        attn_out = self.attention(lstm_out)

        # Global temporal pooling
        pooled = torch.mean(attn_out, dim=1)

        # Apply normalization and dropout
        pooled = self.norm(pooled)
        pooled = self.dropout(pooled)

        # Final classification
        x = self.fc(pooled)

        return x


In [6]:

class ViolenceDetectionModel(nn.Module):
    def __init__(self, feature_dim=768):  # Updated feature dim for ConvNeXt tiny
        super(ViolenceDetectionModel, self).__init__()
        self.feature_extractor = FeatureExtractor(pretrained=True)
        self.feature_dim = feature_dim
        self.sequence_model = TemporalSequenceModel(input_dim=feature_dim)
        
       

        # Freeze feature extractor layers for initial training
        for param in self.feature_extractor.parameters():
            param.requires_grad = False

    def forward(self, x):
        features = self.feature_extractor(x)
        prediction = self.sequence_model(features)
        return prediction

    # Progressive unfreezing to reduce memory impact
    def unfreeze_last_n_layers(self, n=3):
        """Unfreeze only the last n layers of the feature extractor"""
        total_layers = len(list(self.feature_extractor.model.children()))
        print(f"Unfreezing last {n} of {total_layers} feature extractor layers")

        for i, child in enumerate(self.feature_extractor.model.children()):
            if i >= total_layers - n:
                for param in child.parameters():
                    param.requires_grad = True
            else:
                for param in child.parameters():
                    param.requires_grad = False

def create_data_loaders(video_paths_train, labels_train, video_paths_test, labels_test, batch_size=BATCH_SIZE):
    train_dataset = ViolenceDataset(video_paths_train, labels_train, is_train=True)
    test_dataset = ViolenceDataset(video_paths_test, labels_test, is_train=False)

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=custom_collate,
        num_workers=2,  # Reduced workers
        pin_memory=True,
        drop_last=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=custom_collate,
        num_workers=2,  # Reduced workers
        pin_memory=True
    )

    return train_loader, test_loader

def train(model, train_loader, test_loader, epochs=EPOCHS):
    model.to(device)

    # Initialize mixed precision training
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

    # Class weights (if needed)
    class_weights = torch.tensor([1.0, 1.0], device=device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    
    # Progressive unfreezing schedule
    unfreeze_schedule = {
        10: 1,  # At epoch 10, unfreeze last layer
        15: 2,  # At epoch 15, unfreeze last 2 layers
        20: 3   # At epoch 20, unfreeze last 3 layers
    }

    # Optimizer with differential learning rates
    optimizer = optim.AdamW([
        {'params': model.sequence_model.parameters(), 'lr': LEARNING_RATE},
        {'params': model.feature_extractor.parameters(), 'lr': LEARNING_RATE/10}
    ], weight_decay=1e-4)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=5, verbose=True
    )

    best_val_f1 = 0.0
    patience_counter = 0

    # Checkpoint handling
    start_epoch = 0
    if os.path.exists('last_checkpoint.pth'):
        checkpoint = torch.load('last_checkpoint.pth')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_val_f1 = checkpoint.get('f1', 0.0)
        print(f"\nResuming training from epoch {start_epoch}")
        print(f"Previous best F1: {best_val_f1:.4f}")

    print(f"Training on {device}")
    print_gpu_memory()

    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': [],
        'val_f1': []
    }

    try:
        for epoch in range(start_epoch, epochs):
            # Progressive unfreezing
            if epoch in unfreeze_schedule:
                model.unfreeze_last_n_layers(unfreeze_schedule[epoch])
                print_gpu_memory()

            # Training phase
            model.train()
            total_loss = 0
            correct = 0
            total = 0

            print(f"Epoch {epoch+1}/{epochs}")

            for i, batch in enumerate(train_loader):
                try:
                    inputs, labels = batch
                    inputs, labels = inputs.to(device), labels.to(device)

                    # Mixed precision training
                    with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    optimizer.zero_grad()
                    scaler.scale(loss).backward()
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                    scaler.step(optimizer)
                    scaler.update()

                    total_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    correct += (predicted == labels).sum().item()
                    total += labels.size(0)

                    # Print progress and monitor memory
                    if (i+1) % 10 == 0:
                        print(f"  Batch {i+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
                        if (i+1) % 50 == 0:
                            print_gpu_memory()
                            clear_gpu_memory()

                except Exception as e:
                    print(f"Error in batch {i}: {str(e)}")
                    # Save checkpoint before continuing
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': total_loss/(i+1),
                        'f1': best_val_f1
                    }, 'last_checkpoint.pth')
                    continue

            # Save checkpoint at end of each epoch
            avg_loss = total_loss / len(train_loader)
            accuracy = correct / total
            history['train_loss'].append(avg_loss)
            history['train_acc'].append(accuracy)

            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': avg_loss,
                'f1': best_val_f1
            }, 'last_checkpoint.pth')

            # Validation phase
            print("Starting validation...")
            val_metrics = evaluate(model, test_loader, criterion)
            val_loss = val_metrics['loss']
            val_accuracy = val_metrics['accuracy']
            val_f1 = val_metrics['f1']

            # Update history
            history['val_loss'].append(val_loss)
            history['val_acc'].append(val_accuracy)
            history['val_f1'].append(val_f1)

            # Update learning rate and check early stopping
            scheduler.step(val_f1)
            if val_f1 > best_val_f1:
                best_val_f1 = val_f1
                patience_counter = 0
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': val_loss,
                    'accuracy': val_accuracy,
                    'f1': val_f1
                }, 'best_violence_detector.pth')
            else:
                patience_counter += 1
                if patience_counter >= PATIENCE:
                    print(f"Early stopping triggered after {epoch+1} epochs")
                    break

    except KeyboardInterrupt:
        print("\nTraining interrupted by user")
        # Save the current state before exiting
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss if 'avg_loss' in locals() else None,
            'f1': best_val_f1
        }, 'interrupted_checkpoint.pth')
        print("Saved interrupted training state to 'interrupted_checkpoint.pth'")
        return history

    return history

In [7]:

def evaluate(model, data_loader, criterion=None):
    """
    Evaluate the model on the provided data loader.
    Returns a dictionary with various performance metrics.
    """
    model.eval()

    if criterion is None:
        criterion = nn.CrossEntropyLoss()

    total_loss = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for batch in data_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            # Use mixed precision for inference
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            total_loss += loss.item()

            # Get predictions and probabilities
            probs = F.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())

            # Free memory
            del inputs, outputs, labels

    # Convert to numpy arrays for metrics calculation
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)

    # Calculate metrics
    accuracy = (all_preds == all_labels).mean()

    # Get classification report
    report = classification_report(all_labels, all_preds,
                                  target_names=['Non-Violent', 'Violent'],
                                  output_dict=True)

    # Extract F1 score for the positive class (violence)
    f1 = report['Violent']['f1-score']

    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)

    # Calculate ROC and AUC
    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    roc_auc = auc(fpr, tpr)

    return {
        'loss': total_loss / len(data_loader),
        'accuracy': accuracy,
        'f1': f1,
        'report': report,
        'confusion_matrix': cm,
        'predictions': all_preds,
        'labels': all_labels,
        'probabilities': all_probs,
        'roc': {'fpr': fpr, 'tpr': tpr, 'auc': roc_auc}
    }

In [8]:
def plot_training_history(history):
    """
    Plot training and validation metrics.
    """
    plt.figure(figsize=(12, 10))

    # Plot loss
    plt.subplot(2, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy
    plt.subplot(2, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot F1 score
    plt.subplot(2, 2, 3)
    plt.plot(history['val_f1'], label='Validation F1 Score')
    plt.title('F1 Score Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()


In [9]:
def plot_confusion_matrix(cm, classes=['Non-Violent', 'Violent']):
    """
    Plot confusion matrix as a heatmap.
    """
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()

def plot_roc_curve(fpr, tpr, roc_auc):
    """
    Plot ROC curve.
    """
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.tight_layout()
    plt.savefig('roc_curve.png')
    plt.close()


In [ ]:

def create_results_zip():
    """
    Create a ZIP file with all results and visualizations.
    """
    import zipfile

    # Files to include in the ZIP
    result_files = [
        'best_violence_detector.pth',
        'training_history.png',
        'confusion_matrix.png',
        'roc_curve.png',
        # Add any other files you want to include
    ]

    # Create the ZIP file
    with zipfile.ZipFile('results.zip', 'w') as zipf:
        for file in result_files:
            if os.path.exists(file):
                zipf.write(file)

    print("Results ZIP file created successfully.")

import zipfile
import os
from datetime import datetime

def main():
    print("Preparing dataset...")
    (
        video_paths_train, video_paths_test, 
        labels_train, labels_test,
        holdout_paths, holdout_labels
    ) = prepare_dataset()

    # Create ZIP file for holdout videos
    print("\nCreating holdout videos archive...")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_filename = f"holdout_videos_{timestamp}.zip"
    
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for video_path in holdout_paths:
            zipf.write(video_path, os.path.basename(video_path))
    print(f"Saved holdout videos to {zip_filename}")

    print("\nCreating data loaders...")
    train_loader, test_loader = create_data_loaders(
        video_paths_train, labels_train,
        video_paths_test, labels_test
    )

    print("Initializing model...")
    model = ViolenceDetectionModel()

    # Print model summary
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\nModel Summary:")
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")

    # Check initial GPU memory usage
    print_gpu_memory()

    print("\nStarting training...")
    history = train(model, train_loader, test_loader)
    print("Training complete!")

    # Load best model for evaluation
    print("\nEvaluating best model on test set...")
    best_model = ViolenceDetectionModel()
    checkpoint = torch.load('best_violence_detector.pth')
    best_model.load_state_dict(checkpoint['model_state_dict'])
    best_model.to(device)

    # Evaluate on main test set
    test_metrics = evaluate(best_model, test_loader)
    print(f"\nMain Test Set Performance:")
    print(f"Accuracy: {test_metrics['accuracy']:.2%}")
    print(f"F1 Score: {test_metrics['f1']:.4f}")
    print(f"ROC AUC: {test_metrics['roc']['auc']:.4f}")

    # Evaluate on holdout set
    print("\nEvaluating on holdout set...")
    holdout_loader = create_holdout_loader(holdout_paths, holdout_labels)
    holdout_metrics = evaluate(best_model, holdout_loader)
    print(f"\nHoldout Set Performance:")
    print(f"Accuracy: {holdout_metrics['accuracy']:.2%}")
    print(f"F1 Score: {holdout_metrics['f1']:.4f}")
    print(f"ROC AUC: {holdout_metrics['roc']['auc']:.4f}")

    # Save comprehensive report
    print("\nSaving results...")
    save_results(
        history=history,
        test_metrics=test_metrics,
        holdout_metrics=holdout_metrics,
        model=best_model,
        zip_filename=zip_filename
    )

    # Plot results
    plot_training_history(history)
    plot_comparison(test_metrics, holdout_metrics)
    plot_confusion_matrix(holdout_metrics['confusion_matrix'], title="Holdout Set Confusion Matrix")

    return best_model, history, (test_metrics, holdout_metrics)

def create_holdout_loader(video_paths, labels):
    """Create a DataLoader for the holdout set"""
    holdout_dataset = VideoDataset(video_paths, labels)
    return DataLoader(
        holdout_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS
    )

def save_results(history, test_metrics, holdout_metrics, model, zip_filename):
    """Save all results to a structured directory"""
    results_dir = "training_results"
    os.makedirs(results_dir, exist_ok=True)
    
    # Save metrics
    with open(f"{results_dir}/performance_report.txt", "w") as f:
        f.write("=== MAIN TEST SET ===\n")
        f.write(f"Accuracy: {test_metrics['accuracy']:.2%}\n")
        f.write(f"F1: {test_metrics['f1']:.4f}\n")
        f.write(f"ROC AUC: {test_metrics['roc']['auc']:.4f}\n\n")
        f.write("=== HOLDOUT SET ===\n")
        f.write(f"Accuracy: {holdout_metrics['accuracy']:.2%}\n")
        f.write(f"F1: {holdout_metrics['f1']:.4f}\n")
        f.write(f"ROC AUC: {holdout_metrics['roc']['auc']:.4f}\n")
    
    # Save classification reports
    pd.DataFrame(test_metrics['report']).to_csv(f"{results_dir}/test_set_classification_report.csv")
    pd.DataFrame(holdout_metrics['report']).to_csv(f"{results_dir}/holdout_set_classification_report.csv")
    
    # Save confusion matrices
    np.savetxt(f"{results_dir}/test_set_confusion_matrix.csv", test_metrics['confusion_matrix'], delimiter=",")
    np.savetxt(f"{results_dir}/holdout_set_confusion_matrix.csv", holdout_metrics['confusion_matrix'], delimiter=",")
    
    # Copy holdout videos zip
    shutil.copy(zip_filename, results_dir)
    
    print(f"All results saved to {results_dir}/")

if __name__ == "__main__":
    best_model, training_history, metrics = main()

Preparing dataset...
Violent videos per path:
- /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence: 1000 videos
- /kaggle/input/project-data/Complete Dataset/train/Fight: 1000 videos
- /kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Test/Violence: 118 videos
- /kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Train/Violence: 970 videos
- /kaggle/input/project-data/Complete Dataset/val/Fight: 275 videos

Non-violent videos per path:
- /kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence: 1000 videos
- /kaggle/input/project-data/Complete Dataset/train/NonFight: 1000 videos
- /kaggle/input/project-data/Complete Dataset/val/NonFight: 275 videos
- /kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_converted_sec_split/Test/Normal: 169 videos
- /kaggle/input/smartcity-cctv-violence-detection-dataset-scvd/SCVD/SCVD_co

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:00<00:00, 164MB/s]  



Model Summary:
Total parameters: 30,184,802
Trainable parameters: 2,366,210
GPU Memory: Total=14.74GB | Reserved=0.00GB | Allocated=0.00GB | Free=14.74GB

Starting training...
Training on cuda
GPU Memory: Total=14.74GB | Reserved=0.14GB | Allocated=0.11GB | Free=14.60GB
Epoch 1/100


/tmp/ipykernel_31/4126126155.py:62: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/tmp/ipykernel_31/4126126155.py:134: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
/tmp/ipykernel_31/169455789.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):  # Use mixed precision


  Batch 10/661, Loss: 0.9940
  Batch 20/661, Loss: 0.6970
  Batch 30/661, Loss: 0.6456
  Batch 40/661, Loss: 0.5927
  Batch 50/661, Loss: 0.6321
GPU Memory: Total=14.74GB | Reserved=0.23GB | Allocated=0.20GB | Free=14.51GB
  Batch 60/661, Loss: 0.6241
  Batch 70/661, Loss: 0.4130
  Batch 80/661, Loss: 0.8190
  Batch 90/661, Loss: 0.5242
  Batch 100/661, Loss: 0.4569
GPU Memory: Total=14.74GB | Reserved=0.23GB | Allocated=0.20GB | Free=14.51GB
  Batch 110/661, Loss: 0.3098
  Batch 120/661, Loss: 0.4896
  Batch 130/661, Loss: 0.6788
  Batch 140/661, Loss: 0.4292
  Batch 150/661, Loss: 0.4810
GPU Memory: Total=14.74GB | Reserved=0.23GB | Allocated=0.20GB | Free=14.51GB
  Batch 160/661, Loss: 0.4013
  Batch 170/661, Loss: 0.2721
  Batch 180/661, Loss: 0.3741
  Batch 190/661, Loss: 0.3125
  Batch 200/661, Loss: 0.3501
GPU Memory: Total=14.74GB | Reserved=0.23GB | Allocated=0.20GB | Free=14.51GB
  Batch 210/661, Loss: 0.2484
  Batch 220/661, Loss: 0.5272
  Batch 230/661, Loss: 0.3967
  Batch

...

Epoch 39/100


/tmp/ipykernel_31/4126126155.py:134: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
/tmp/ipykernel_31/169455789.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True):  # Use mixed precision


  Batch 10/661, Loss: 0.0000
  Batch 20/661, Loss: 0.0000
  Batch 30/661, Loss: 0.0000
  Batch 40/661, Loss: 0.0000
  Batch 50/661, Loss: 0.0000
GPU Memory: Total=14.74GB | Reserved=5.54GB | Allocated=0.52GB | Free=9.20GB
  Batch 60/661, Loss: 0.0000
  Batch 70/661, Loss: 0.0000
  Batch 80/661, Loss: 0.0000
  Batch 90/661, Loss: 0.0000
  Batch 100/661, Loss: 0.0000
GPU Memory: Total=14.74GB | Reserved=5.54GB | Allocated=0.52GB | Free=9.20GB
  Batch 110/661, Loss: 0.0000
  Batch 120/661, Loss: 0.0000
  Batch 130/661, Loss: 0.9170
  Batch 140/661, Loss: 0.0000
  Batch 150/661, Loss: 0.0000
GPU Memory: Total=14.74GB | Reserved=5.54GB | Allocated=0.52GB | Free=9.20GB
  Batch 160/661, Loss: 0.0000
  Batch 170/661, Loss: 0.0000


[h264 @ 0x5fbcd5c0] mb_type 104 in P slice too large at 98 31
[h264 @ 0x5fbcd5c0] error while decoding MB 98 31


  Batch 180/661, Loss: 0.0001
  Batch 190/661, Loss: 0.0000
  Batch 200/661, Loss: 0.0000
GPU Memory: Total=14.74GB | Reserved=5.54GB | Allocated=0.52GB | Free=9.20GB
  Batch 210/661, Loss: 0.0000
  Batch 220/661, Loss: 0.0000
  Batch 230/661, Loss: 0.0000
  Batch 240/661, Loss: 0.0000
  Batch 250/661, Loss: 0.0000
GPU Memory: Total=14.74GB | Reserved=5.54GB | Allocated=0.52GB | Free=9.20GB
  Batch 260/661, Loss: 0.0000
  Batch 270/661, Loss: 0.0000
  Batch 280/661, Loss: 0.0000
  Batch 290/661, Loss: 0.0000
  Batch 300/661, Loss: 0.0000
GPU Memory: Total=14.74GB | Reserved=5.54GB | Allocated=0.52GB | Free=9.20GB
  Batch 310/661, Loss: 0.0000
  Batch 320/661, Loss: 0.0000
  Batch 330/661, Loss: 0.7705
  Batch 340/661, Loss: 0.0000
  Batch 350/661, Loss: 0.0000
GPU Memory: Total=14.74GB | Reserved=5.54GB | Allocated=0.52GB | Free=9.20GB
  Batch 360/661, Loss: 0.0000
  Batch 370/661, Loss: 0.0000
  Batch 380/661, Loss: 0.0000
  Batch 390/661, Loss: 0.0000
  Batch 400/661, Loss: 0.0004
GP